# Snapshot Volatility Tool — Demo

This notebook demonstrates the `SnapshotVolatilityTool`, the first deterministic
tool module in the prediction_agent framework.

It computes five market behaviour metrics from local JSONL snapshot data:
1. **Volatility** — standard deviation of price series
2. **Price range** — max − min price
3. **Mean spread** — average (ask − bid)
4. **Jump rate** — fraction of consecutive moves > 0.05
5. **Liquidity proxy** — mean open_interest (or volume)


In [ ]:
import sys, json
from pathlib import Path
from datetime import datetime, timezone

# Ensure prediction_agent is importable
# notebooks/ sits inside prediction_agent/, so go up TWO levels
# to reach the directory that contains the prediction_agent package.
PROJECT_ROOT = Path.cwd().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

PREDICTION_AGENT_DIR = PROJECT_ROOT / "prediction_agent"
print(f"Project root: {PROJECT_ROOT}")
print(f"Package dir : {PREDICTION_AGENT_DIR}")

## 1. Load the dataset

In [ ]:
JSONL_PATH = PREDICTION_AGENT_DIR / "outputs" / "market_snapshots.jsonl"

rows = []
with open(JSONL_PATH, "r") as f:
    for line in f:
        line = line.strip()
        if line:
            rows.append(json.loads(line))

print(f"Total rows: {len(rows)}")
print(f"Unique market_ids: {len(set(r['market_id'] for r in rows))}")

# Show market_id counts
from collections import Counter
counts = Counter(r["market_id"] for r in rows)
print("\nTop 10 markets by row count:")
for mid, cnt in counts.most_common(10):
    print(f"  {mid}: {cnt} rows")

## 2. Pick a market and run the tool

In [ ]:
from prediction_agent.schemas import EventInput
from prediction_agent.tools.snapshot_volatility_tool import SnapshotVolatilityTool

# Pick the market with the most rows
target_market_id = counts.most_common(1)[0][0]
print(f"Selected market: {target_market_id}")

# Create an EventInput
event = EventInput(
    event_id="demo-event",
    market_id=target_market_id,
    market_title="Demo",
    current_price=0.50,
)

# Run the tool (uses a very wide window so all data is included)
tool = SnapshotVolatilityTool(jsonl_path=JSONL_PATH)
result = tool.run(event, window_minutes=999_999)

print("\n=== Tool Output ===")
print(f"tool_name    : {result.tool_name}")
print(f"output_vector: {result.output_vector}")
print(f"confidence   : {result.metadata['confidence']}")
print(f"sample_count : {result.metadata['sample_count']}")

labels = ["volatility", "price_range", "mean_spread", "jump_rate", "liquidity_proxy"]
print("\nBreakdown:")
for label, val in zip(labels, result.output_vector):
    print(f"  {label:20s} = {val}")

## 3. Plot price vs time

In [ ]:
import matplotlib
matplotlib.use("Agg")          # non-interactive backend
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Filter rows for the target market
market_rows = [r for r in rows if r["market_id"] == target_market_id]

# Extract timestamps and prices
timestamps = []
prices = []
for r in market_rows:
    ts = datetime.fromisoformat(r["timestamp"])
    lp = r.get("last_price")
    bid = r.get("yes_bid")
    ask = r.get("yes_ask")
    if lp is not None and lp > 0:
        price = lp
    elif bid is not None and ask is not None and (bid > 0 or ask > 0):
        price = (bid + ask) / 2.0
    else:
        continue
    timestamps.append(ts)
    prices.append(price)

# Sort by time
pairs = sorted(zip(timestamps, prices), key=lambda x: x[0])
timestamps = [p[0] for p in pairs]
prices = [p[1] for p in pairs]

# Plot
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(timestamps, prices, marker="o", linewidth=1.5, markersize=4, color="#2563eb")
ax.set_title(f"Price vs Time — {target_market_id}", fontsize=14)
ax.set_xlabel("Timestamp (UTC)")
ax.set_ylabel("Price")
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fig.autofmt_xdate()
ax.grid(True, alpha=0.3)

# Add volatility annotation
vol = result.output_vector[0]
pr  = result.output_vector[1]
ax.annotate(
    f"σ = {vol:.4f}\nrange = {pr:.4f}",
    xy=(0.02, 0.95), xycoords="axes fraction",
    fontsize=10, verticalalignment="top",
    bbox=dict(boxstyle="round,pad=0.4", facecolor="lightyellow", alpha=0.8),
)

plt.tight_layout()
plt.savefig(str(PREDICTION_AGENT_DIR / "outputs" / "price_vs_time.png"), dpi=150)
plt.show()
print("Plot saved to outputs/price_vs_time.png")

## 4. Registry integration check

In [ ]:
from prediction_agent.tools.registry import build_default_registry

registry = build_default_registry()
print("Registered tools:")
for t in registry.list_tools():
    print(f"  • {t['name']}: {t['description'][:80]}...")

assert "snapshot_volatility_tool" in registry
print("\n✓ snapshot_volatility_tool is registered and ready.")